In [1]:
from datetime import datetime as dt
print('Last accessed on: {}'.format(dt.now()))

Last accessed on: 2017-06-22 19:27:02.800658


In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Exoplanet plots

# To Do: 
* Plots (with uncertainty bars)
  * reproduce [these plots](http://exoplanetarchive.ipac.caltech.edu/exoplanetplots/) and [these](http://seagerexoplanets.mit.edu/research.htm) and may be some from [here](http://th.nao.ac.jp/MEMBER/hori/pdf/HORI_2012Dec3-5.pdf); note also useful [formula](http://exoplanetarchive.ipac.caltech.edu/docs/poet_calculations.html#insol_flux); save in folder: `C:\Users\Jerome\Box Sync\research papers\images\plots`
    * M vs P
    * R vs P
    * R vs P
    * e vs P
    * Irradiation (Insolation flux) vs P
    * R vs M → density (solid, rocky, icy, thick atmosphere?)
    * discovery per year
    * Earth-like planets (HZ shown)
* possible grouping
  * detection method
  * hot Jupiters
  * mini-Neptunes

* Use clustering algorithm to get planet groups (perhaps refers to distinct planet regimes)
* Get mass radius relation for each planet grouping
  * Different relations applies for different planet regimes (close-in, far-out)?
* Extrapolate the mass of the planet for each regime 
* Earth-like planets accdg by filtering the ff params:
  * radius, temperature, bulk composition, stellar host type, rotation rate, companion moon, big brother gas giant planet, geologic activity, existence of water, atmosphere composition
* Regime convergence (transit+RV, imaging+RV, microlensing+RV+transit)
  * Overlap between RV and Transiting planets
* Use Neural Nets to model the distribution of directly imaged exoplanets and compare it to the empirically-determined distribution for RV planets (see Brandt 2014)
* Check plot semi-major axis vs. planetary radius and look for far-away (or long period) super-Earth/mini-Neptunes that can be interesting targets for or direct imaging
* Super-Earths or mini-Neptune?
* Extreme planets 

DONE
* Kepler's third law in transiting planets
* Multiplanet systems trends and distribution
  * Explore the architecture in multiple planetary systems (see Hot Jupiters.ipynb)
    * https://en.wikipedia.org/wiki/List_of_multiplanetary_systems


# Objective
The objective of this notebook is to query data from NASA exoplanet archive and plot all interesting trends, relations, distributions of exoplanet, and add some statistical analysis

# Data
The NASA Exoplanet Archive provides data and various tools related to exoplanets. See for example the [table of confirmed exoplanet data](http://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=planets) or list of [data catalog](http://exoplanetarchive.ipac.caltech.edu/docs/data.html).

While we can make scatter plots and histograms in the website, we would like to play with the data and do some advanced visualization and analysis. Thanks to the provision of the API, we can query data directly from their [website]( 
http://exoplanetarchive.ipac.caltech.edu/docs/program_interfaces.html). See advanced querying using their [API](http://exoplanetarchive.ipac.caltech.edu/docs/program_interfaces.html).

In [22]:
# Python 2 and 3 compatible
try:
    from urllib.request import urlopen, Request
    from urllib.error import HTTPError
except ImportError:
    from urllib2 import urlopen, Request, HTTPError
import time
import warnings
warnings.filterwarnings('ignore')


url = 'http://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?table=exoplanets'
pl_params = "pl_hostname,pl_letter,pl_discmethod,pl_disc,pl_pnum,pl_orbper,pl_orbsmax,pl_orbeccen,pl_orbincl,pl_bmassj,pl_radj,pl_radjerr1,pl_radjerr2,pl_dens,pl_instrument,pl_disc_refname,pl_pelink,pl_insol,pl_eqt,"
pl_err = 'pl_bmassjerr1,pl_bmassjerr2,pl_bmassjerr1,pl_bmassjerr2,pl_orbpererr1,pl_orbpererr2,'
st_params = "ra,dec,st_dist,st_teff,st_mass,st_rad,st_plx,gaia_plx,gaia_dist,st_pm,gaia_pm,st_radv,st_spstr,st_logg,st_lum,st_dens,st_metfe,st_age,st_acts,st_bmvj,st_jmh2,"
others = 'pl_instrument,pl_telescope,rowupdate,st_nplc'
full_url= url+'&select='+ pl_params + pl_err + st_params+others

response = urlopen(full_url)
html = response.read()

outpath_extended = 'confirmed_planets_extended_{}.csv'.format(time.strftime("%Y%m%d")) #include date of download

print("Downloading data from URL:\n{}".format(url))

with open(outpath_extended,'wb') as f:
     f.write(html)
print("Saved file as {}".format(outpath_extended))

http://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?table=exoplanets
Saved file as confirmed_planets_extended_20170622.csv


Let's check the downloaded file.

In [23]:
html[:100]

b'pl_hostname,pl_letter,pl_discmethod,pl_disc,pl_pnum,pl_orbper,pl_orbsmax,pl_orbeccen,pl_orbincl,pl_b'

Or better yet, parse into table using pandas: 

In [24]:
import pandas as pd
#default
df = pd.read_csv(outpath_extended)
df.head(3)

,pl_hostname,pl_letter,pl_discmethod,pl_disc,pl_pnum,pl_orbper,pl_orbsmax,pl_orbeccen,pl_orbincl,pl_bmassj,...,st_dens,st_metfe,st_age,st_acts,st_bmvj,st_jmh2,pl_instrument.1,pl_telescope,rowupdate,st_nplc
0,HD 240237,b,Radial Velocity,2011,1,745.7000,1.90,0.400,NaN,5.300,...,NaN,-0.26,NaN,NaN,1.682,0.633,High Resolution Spectrograph,9.2 m Hobby-Eberly Telescope,2014-11-19,0
1,HD 290327,b,Radial Velocity,2009,1,2443.0000,3.43,0.080,NaN,2.540,...,NaN,-0.11,NaN,NaN,0.761,0.279,HARPS Spectrograph,3.6 m ESO Telescope,2014-11-19,0
2,HD 285507,b,Radial Velocity,2013,1,6.0881,NaN,0.086,NaN,0.917,...,NaN,0.13,0.625,NaN,1.415,0.554,TRES Echelle Spectrograph,1.5 m Tillinghast Reflector Telescope,2014-05-14,0


### Hot Jupiters

[Hot Jupiters](https://en.wikipedia.org/wiki/Hot_Jupiter) are a rare and peculiar class of exoplanet that have masses comparable to that of Jupiter ($\gtrsim$0.1 M$_{Jup}$) on orbits with periods $P$ less than about 10 days (e.g., Gaudi et al. 2005; Wright et al. 2012).

In [ ]:
df_hot_jup = df.query('0.36 <= pl_bmassj <= 11.8 and pl_orbper <= 10')
df_RV_hot_jup = df.query('0.36 <= pl_bmassj <= 11.8 and pl_orbper <= 10')
df_transit_hot_jup = df.query('0.36 <= pl_bmassj <= 11.8 and pl_orbper <= 10')
df_jup_like = df.query('0.5 <= pl_bmassj <= 11.8 and 1 <= pl_orbsmax <= 10')

[Ngo et al. 2015](http://adsabs.harvard.edu/cgi-bin/bib_query?arXiv:1501.00013):
>Surveys from the past few years (e.g. Winn et al. 2010; Albrecht et al. 2012) indicate misaligned hot Jupiters are common--18 out of 53 hot Jupiters surveyed to date have [obliquities](https://en.wikipedia.org/wiki/Axial_tilt) that are inconsistent with zero at the [three sigma level](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule) or higher. As a result, it has been arued that a significant fraction of hot Jupiters may have migrated via three-body interactions such as the [Kozai-Lidov mechanism](https://en.wikipedia.org/wiki/Kozai_mechanism) (c.f. [simulation](http://users.monash.edu.au/~dprice/pubs/kozai/)), which naturally results in large orbital inclinations (e.g.  Morton & Johnson 2011; Li et al. 2014). If stellar tides can bring misaligned hot Jupiters back into alignment with the star’s spin axis (Dawson 2014), this fraction may be even higher than the current rate suggests. 

In [ ]:
df_hot_jup.describe()

In [ ]:
len(df_hot_jup['pl_orbincl'].dropna())

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

#note: this is inclination, not obliquity 
plt.hist(df['pl_orbincl'].dropna(), bins = 20, label='all', normed=True)
plt.hist(df_hot_jup['pl_orbincl'].dropna(), bins = 20, label='HJ', normed=True, alpha=0.5)
plt.legend(loc='best')

In [ ]:
import numpy as np

mean= np.mean(df_hot_jup['pl_orbincl'].dropna())
print('mean= {}'.format(mean))

#df_hot_jup['pl_orbincl'].std(axis=0,skipna=True)
sigma =np.std(df_hot_jup['pl_orbincl'].dropna())
print('sigma= {}'.format(sigma))

#df_hot_jup['pl_orbincl'].dropna().query('pl_orbincl > threesigma')

In [ ]:
#inclination greater than 3sigma

#one-liner
#[i for i in df_hot_jup['pl_orbincl'].dropna().values if abs(mean-i) > 3*sigma]
for i in df_hot_jup['pl_orbincl'].dropna().values:
    if abs(mean-i) > 3*sigma:
        print(i)

In [ ]:
#for hot jupiters; see df
inc=df_hot_jup['pl_orbincl'].dropna()
inc[((inc-inc.mean()).abs()>3*inc.std())]#.index

In [ ]:
len(inc[((inc-inc.mean()).abs()>3*inc.std())])/float(len(inc))*100

In [ ]:
#generally for all planets
inc2=df['pl_orbincl'].dropna()
len(inc2[((inc2-inc2.mean()).abs()>3*inc2.std())])/float(len(inc2))*100

In [ ]:
#compute z>3 for all float columns
from scipy import stats

float_values=df_hot_jup.select_dtypes(include=[float])

#z>0
df_hot_jup[(np.abs(stats.zscore(float_values) > 3))]
#df_hot_jup[(np.abs(stats.zscore(df_hot_jup)) < 3).all(axis=1)]

#see also 2nd answer http://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-dataframe

>Conversely, Dawson et al. (2015) argue that the lack of high-eccentricity Jupiters at intermediate periods in the overall Kepler sample places a strict upper limit on the fraction of hot Jupiters that might have migrated via three-body processes.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

#note: this is inclination, not obliquity 
#plt.hist(df['pl_orbsmax'].dropna(), bins = 20, label='all', normed=True)
plt.hist(df_transit_hot_jup['pl_orbsmax'].dropna(), bins = 20, label='HJ', normed=True, alpha=0.5)
plt.legend(loc='best')

In [ ]:
fig, ax = plt.subplots()
df.plot('pl_orbsmax', 'pl_orbeccen', kind='scatter', label='all', color='red', alpha=0.5, logx=True, logy=False, ax=ax, title='All Hot Jupiters')
df_hot_jup.plot('pl_orbsmax', 'pl_orbeccen', kind='scatter', label='HJ', logx=True, logy=False, ax=ax)#, loglog=True)# doesn't work well

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

#df[['x', 'y']].dropna() makes sure pair has equal len
x=df_transit[['pl_orbsmax', 'pl_orbeccen']].dropna()['pl_orbsmax'].values
y=df_transit[['pl_orbsmax', 'pl_orbeccen']].dropna()['pl_orbeccen'].values

# Calculate the point density
xy = np.vstack([x,y])
z = gaussian_kde(xy)(xy)

fig, ax = plt.subplots()
ax.scatter(x, y, c=z, s=100, edgecolor='')
#ax.set_xscale('log')

In [ ]:
fig, ax = plt.subplots()
df_transit.plot('pl_orbsmax', 'pl_orbeccen', kind='scatter', label='all', color='red', alpha=0.5, logx=True, logy=False, ax=ax, title='Transiting Planets')
df_transit_hot_jup.plot('pl_orbsmax', 'pl_orbeccen', kind='scatter', label='HJ', logx=True, logy=False, ax=ax)#, loglog=True)# doesn't work well
#density plot seems not scaled with two plots above
#df_transit_hot_jup.plot.density(x='pl_orbsmax', y='pl_orbeccen', c='b', ax=ax);

In [ ]:
#fraction of hot jupiters
len(df_transit_hot_jup['pl_orbeccen'].dropna())/float(len(df_transit['pl_orbeccen'].dropna()))*100

>Misaligned hot Jupiters may also result from migration in a tilted disk, which could be caused by torque from a distant stellar companion (Batygin 2012). Moreover, significant star-disk misalignments may naturally arise from the physical evolution of the star and the disk in a perturbed system (Batygin & Adams 2013; Spalding & Batygin 2014a). This suggests that a hot Jupiter’s obliquity, which can be measured with the Rossiter–McLaughlin effect (Winn et al. 2005) or via Doppler tomography (e.g., Collier Cameron et al. 2010; Brown et al. 2012), might provide a clue to whether or not a third body has influenced the planetary system.

Number of planets in a binary system

In [ ]:
#plt.hist(df.query('pl_cbflag == 1')['pl_pnum'], bins=[1,2,3,4,5])
df.query('pl_cbflag == 1')['pl_pnum'].plot(kind='hist')

#### RV planet distribution

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

#plt.hist(df_RV['pl_orbsmax'].dropna(), bins=40)

import numpy as np
import matplotlib.pyplot as plt
from astroML.plotting import hist
#x = np.random.normal(size=1000)
x = df_RV['pl_orbsmax'].dropna().values
x2 = df_RV_hot_jup['pl_orbsmax'].dropna().values

fig = plt.figure(figsize=(10, 4))
fig.subplots_adjust(left=0.1, right=0.95, bottom=0.15)

for bins, title, subplot in zip(['knuth', 'blocks'],
                                ["Knuth's rule", 'Bayesian blocks'],
                                [121, 122]):
    ax = fig.add_subplot(subplot)

    # plot a standard histogram in the background, with alpha transparency
    hist(np.log(x), bins=100, histtype='stepfilled',
         alpha=0.2, normed=True, label='standard histogram')

    # plot an adaptive-width histogram on top
    hist(np.log(x), bins=bins, ax=ax, color='black',
         histtype='step', normed=True, label=title)

    ax.legend(loc='best', prop=dict(size=12))
    ax.set_xlabel('$\log$ Semi-major axis (AU)')
    ax.set_ylabel('Count')

for bins, title, subplot in zip(['knuth', 'blocks'],
                                ["Hot Jupiter (Knuth)", 'Hot Jupiter (Bayesian)'],
                                [121, 122]):
    ax = fig.add_subplot(subplot)

    # plot a standard histogram in the background, with alpha transparency
    hist(np.log(x2), bins=100, histtype='stepfilled',
         alpha=0.2, normed=True, label='Hot Jupiter (standard)')

    # plot an adaptive-width histogram on top
    hist(np.log(x2), bins=bins, ax=ax, color='black',
         histtype='step', normed=True, label=title)

    ax.legend(loc='best', prop=dict(size=12))
    ax.set_xlabel('$\log$ Semi-major axis (AU)')
    ax.set_ylabel('Count')
    
plt.show()

### Transit planet distribution

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

#plt.hist(df_RV['pl_orbsmax'].dropna(), bins=40)

import numpy as np
import matplotlib.pyplot as plt
from astroML.plotting import hist
#x = np.random.normal(size=1000)
x = df_transit['pl_orbsmax'].dropna().values
x2 = df_transit_hot_jup['pl_orbsmax'].dropna().values

fig = plt.figure(figsize=(12, 4))
fig.subplots_adjust(left=0.1, right=0.95, bottom=0.15)

for bins, title, subplot in zip(['knuth', 'blocks'],
                                ["Knuth's rule", 'Bayesian blocks'],
                                [121, 122]):
    ax = fig.add_subplot(subplot)

    # plot a standard histogram in the background, with alpha transparency
    hist(np.log(x), bins=100, histtype='stepfilled',
         alpha=0.2, normed=True, label='standard histogram')

    # plot an adaptive-width histogram on top
    hist(np.log(x), bins=bins, ax=ax, color='black',
         histtype='step', normed=True, label=title)

    ax.legend(loc='best', prop=dict(size=10))
    ax.set_xlabel('$\log$ Semi-major axis (AU)')
    ax.set_ylabel('Count')

for bins, title, subplot in zip(['knuth', 'blocks'],
                                ["Hot Jupiter (Knuth)", 'Hot Jupiter (Bayesian)'],
                                [121, 122]):
    ax = fig.add_subplot(subplot)

    # plot a standard histogram in the background, with alpha transparency
    hist(np.log(x2), bins=100, histtype='stepfilled',
         alpha=0.2, normed=True, label='Hot Jupiter (standard)')

    # plot an adaptive-width histogram on top
    hist(np.log(x2), bins=bins, ax=ax, color='black',
         histtype='step', normed=True, label=title)

    ax.legend(loc='best', prop=dict(size=12))
    ax.set_xlabel('$\log$ Semi-major axis (AU)')
    ax.set_ylabel('Count')
    
plt.show()

### Multiple planet vs Single planet distribution (RV)

> [Figure 5](http://iopscience.iop.org/article/10.1086/659427/pdf) shows that among the multiplanet systems, the semimajor-axis distribution is quite distinct: multiplanet
systems are much less likely to include a close-in planet, and there also does not appear to be a 1-AU jump among the multiplanet systems. 

> This has been interpreted as evidence for migration, and is used as a test for different migration scenarios (e.g., Beaugé & Nesvorný 2012).

In [ ]:
df.query('pl_pnum > 1').head(3)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

#plt.hist(df_RV['pl_orbsmax'].dropna(), bins=40)

import numpy as np
import matplotlib.pyplot as plt
from astroML.plotting import hist
#x = np.random.normal(size=1000)
x = df_RV.query('pl_pnum > 1')
x = x['pl_orbsmax'].dropna().values
x2 = df_RV_hot_jup.query('pl_pnum > 1')
x2 = x2['pl_orbsmax'].dropna().values

fig = plt.figure(figsize=(12, 4))
fig.subplots_adjust(left=0.1, right=0.95, bottom=0.15)

for bins, title, subplot in zip(['knuth', 'blocks'],
                                ["Knuth's rule", 'Bayesian blocks'],
                                [121, 122]):
    ax = fig.add_subplot(subplot)

    # plot a standard histogram in the background, with alpha transparency
    hist(np.log(x), bins=100, histtype='stepfilled',
         alpha=0.2, normed=True, label='standard histogram')

    # plot an adaptive-width histogram on top
    hist(np.log(x), bins=bins, ax=ax, color='black',
         histtype='step', normed=True, label=title)

    ax.legend(loc='best', prop=dict(size=10))
    ax.set_xlabel('$\log$ Semi-major axis (AU)')
    ax.set_ylabel('Count')

for bins, title, subplot in zip(['knuth', 'blocks'],
                                ["Hot Jupiter (Knuth)", 'Hot Jupiter (Bayesian)'],
                                [121, 122]):
    ax = fig.add_subplot(subplot)

    # plot a standard histogram in the background, with alpha transparency
    hist(np.log(x2), bins=100, histtype='stepfilled',
         alpha=0.2, normed=True, label='Hot Jupiter (standard)')

    # plot an adaptive-width histogram on top
    hist(np.log(x2), bins=bins, ax=ax, color='black',
         histtype='step', normed=True, label=title)

    ax.legend(loc='best', prop=dict(size=12))
    ax.set_xlabel('$\log$ Semi-major axis (AU)')
    ax.set_ylabel('Count')
    
plt.show()

---

# Mass-orbital radius distribution

In [ ]:
print(plt.style.available)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')

fig, ax = plt.subplots() #figsize=(10,8)

groups=df.groupby('pl_discmethod')

colors = ["#476A2A", "#7851B8", "#BD3430", "#4A2D4E", "#875525",
          "#A83683", "#4E655E", "#853541", "#3A3120","#535D8E"]

#i=0
for name, group in groups:
    group.plot(x='pl_orbsmax', y='pl_bmassj', kind='scatter', alpha=0.6, label=str(name), ax=ax)#, color=colors[i])
    #i+=1

plt.xscale('log')
plt.yscale('log')
plt.ylabel('Mass [$M_{Jup}$]')
plt.xlabel('Semi-major axis [AU]')
plt.ylabel('Mass [$M_{Jup}$]')
plt.ylabel('Mass [$M_{Jup}$]')
plt.xlim([1e-3, 5e4])
plt.ylim([1e-3, 5e2])

#'''
#make a colormap
colormap = plt.cm.gist_ncar #nipy_spectral, Set1,Paired  
colorst = [colormap(i) for i in np.linspace(0, 0.9,len(ax.collections))]       
for t,j1 in enumerate(ax.collections):
    j1.set_color(colorst[t])
#'''


ax.legend(fontsize='small', loc=4)
plt.show()

In [ ]:
from matplotlib import pylab as pl
%pylab inline

pl.plot(df_transit['pl_orbsmax'],df_transit['pl_bmassj'],'bo', label='transit ({})'.format(np.count_nonzero(df_transit['pl_bmassj']>0)))
pl.plot(df_RV['pl_orbsmax'],df_RV['pl_bmassj'],'ro', label='RV ({})'.format(np.count_nonzero(df_RV['pl_bmassj']>0)))
pl.plot(df_ML['pl_orbsmax'],df_ML['pl_bmassj'],'go', label='microlensing ({})'.format(np.count_nonzero(df_ML['pl_bmassj']>0)))
pl.plot(df_DI['pl_orbsmax'],df_DI['pl_bmassj'],'mo', label='direct imaging ({})'.format(np.count_nonzero(df_DI['pl_bmassj']>0)))
pl.plot(df_TTV['pl_orbsmax'],df_TTV['pl_bmassj'],'yo', label='TTV ({})'.format(np.count_nonzero(df_TTV['pl_bmassj']>0)))
pl.plot(df_OBM['pl_orbsmax'],df_OBM['pl_bmassj'],'co', label='orb. br. mod. ({})'.format(np.count_nonzero(df_OBM['pl_bmassj']>0)))
pl.xlabel('Semi-major axis [AU]')
pl.ylabel('Planet Mass [$M_{Jup}$]')
pl.xlim([5e-3, 5e5])
pl.xscale('log')
pl.yscale('log')
pl.plot(a_E/a_E,M_E/M_J,'k*', markersize=10, label='Earth')
pl.plot(a_J/a_E,M_J/M_J,'ks', markersize=10, label='Jupiter')
pl.plot(a_N/a_E,M_N/M_J,'k^', markersize=10, label='Neptune')
pl.legend(loc=4, numpoints = 1, fontsize=10)
pl.show()
#fname='../mass_SMA_11.13.png'
# pl.savefig(fname, dpi=None, facecolor='w', edgecolor='w',
#         orientation='portrait', papertype=None, format=None,
#         transparent=False, bbox_inches=None, pad_inches=0.1,
#         frameon=None)

In [ ]:
matplotlib.rcParams.update({'font.size': 16})

#Constants that will be useful later
M_E = 5.972e24 #kg
a_E = 149.60e6 #m 
R_E = 6371e3 #km
P_E = 365 #d

M_J = 1.898e27 #kg
a_J = 778.57e6 #m
R_J = 69911e3 #m
P_J = 11.86*P_E

M_N = 1.024e26
a_N = 4495.06e6 #m
R_N = 24622e3 #m
P_N = 164.8*P_E

from matplotlib import pylab as pl
%pylab inline

pl.plot(df_transit['pl_orbsmax'],df_transit['pl_bmassj'],'bo', alpha=0.5, label='transit ({})'.format(np.count_nonzero(df_transit['pl_bmassj']>0)))
pl.plot(df_RV['pl_orbsmax'],df_RV['pl_bmassj'],'ro', alpha=0.5, label='RV ({})'.format(np.count_nonzero(df_RV['pl_bmassj']>0)))
pl.plot(df_ML['pl_orbsmax'],df_ML['pl_bmassj'],'go', alpha=0.5, label='microlensing ({})'.format(np.count_nonzero(df_ML['pl_bmassj']>0)))
pl.plot(df_DI['pl_orbsmax'],df_DI['pl_bmassj'],'mo', alpha=0.5, label='direct imaging ({})'.format(np.count_nonzero(df_DI['pl_bmassj']>0)))
pl.plot(df_TTV['pl_orbsmax'],df_TTV['pl_bmassj'],'yo', alpha=0.5, label='TTV ({})'.format(np.count_nonzero(df_TTV['pl_bmassj']>0)))
pl.plot(df_OBM['pl_orbsmax'],df_OBM['pl_bmassj'],'co', alpha=0.5, label='orb. br. mod. ({})'.format(np.count_nonzero(df_OBM['pl_bmassj']>0)))
pl.xlabel('Semi-major axis [AU]')
pl.ylabel('Planet mass [$M_{J}$]')
pl.xlim([1e-2, 1e5])
pl.xscale('log')
pl.yscale('log')
#pl.plot(a_E/a_E,M_E/M_J,'k*', markersize=15, label='Earth')
#pl.plot(a_J/a_E,M_J/M_J,'kD', markersize=12, label='Jupiter')
#pl.plot(a_N/a_E,M_N/M_J,'k^', markersize=15, label='Neptune')
pl.xlim([1e-3,1e4])
pl.legend(loc=4, numpoints = 1, fontsize=10)
pl.show()

What is that blue dot below the star (Earth)?

In [ ]:
sample2 = df_transit.query('pl_bmassj <= 1e-3 and pl_orbsmax <= 1')
#idx = df_sample2.index
sample2

### With mpdld3 

#### interactive tooltip

In [ ]:
#planets with known orbital a, M, and R
len(df_transit.query('pl_radj > 0 & pl_orbsmax > 0 & pl_bmassj > 0'))

In [ ]:
#to fix log axis format, see: http://nbviewer.jupyter.org/gist/aflaxman/988cb466117430e8ba1b
import matplotlib.pyplot as plt
import numpy as np
import mpld3

fig, ax = plt.subplots(subplot_kw=dict(axisbg='#EEEEEE'))

#cannot use df_transit.plot('pl_orbsmax', 'pl_bmassj', kind='scatter', alpha=0.3)
#too few s=df_transit['pl_radj']*100
trans = ax.scatter(df_transit['pl_orbsmax'],df_transit['pl_bmassj'], alpha=0.3)#, label='transit ({})'.format(np.count_nonzero(df_transit['pl_bmassj']>0)))
#pl.plot(df_RV['pl_orbsmax'],df_RV['pl_bmassj'],'ro', label='RV ({})'.format(np.count_nonzero(df_RV['pl_bmassj']>0)))
#pl.plot(df_ML['pl_orbsmax'],df_ML['pl_bmassj'],'go', label='microlensing ({})'.format(np.count_nonzero(df_ML['pl_bmassj']>0)))
#pl.plot(df_DI['pl_orbsmax'],df_DI['pl_bmassj'],'mo', label='direct imaging ({})'.format(np.count_nonzero(df_DI['pl_bmassj']>0)))
#pl.plot(df_TTV['pl_orbsmax'],df_TTV['pl_bmassj'],'yo', label='TTV ({})'.format(np.count_nonzero(df_TTV['pl_bmassj']>0)))
#pl.plot(df_OBM['pl_orbsmax'],df_OBM['pl_bmassj'],'co', label='orb. br. mod. ({})'.format(np.count_nonzero(df_OBM['pl_bmassj']>0)))

ax.grid(color='white', linestyle='solid')
ax.set_xlabel('Semi-major axis [AU]')
ax.set_ylabel('Planet Mass [M_J]')
ax.set_xlim([5e-3, 5e5])
ax.set_ylim([1e-4, 1e2])
ax.set_xscale('log')
ax.set_yscale('log')
#pl.plot(a_E/a_E,M_E/M_J,'k*', markersize=15, label='Earth')
#pl.plot(a_J/a_E,M_J/M_J,'ks', markersize=12, label='Jupiter')
#pl.plot(a_N/a_E,M_N/M_J,'k^', markersize=15, label='Neptune')

name = list(df_transit.pl_hostname.values)
labels = ['{0}'.format(str(i) for i in name)]
#connect label and plot
tooltip = mpld3.plugins.PointLabelTooltip(trans, labels=labels)
#connect fig and tooltip
mpld3.plugins.connect(fig, tooltip)

mpld3.display()

Things to fix mpld3 rending:
* label does not work: generator ids showup instead of names
* fig zooming is enabled but not in datapoints 

### With bokeh

see exopanet plot (interactive).ipynb, bokeh-notebooks-master/02 - column data source.ipynb

# Density plot
See Seager et al. 2008

# Hot Jupiters
http://www.nature.com/nature/journal/v537/n7621/full/nature19430.html
1. There is an excess of objects that have orbital periods shorter than about 10 days and masses similar to that of Jupiter
2. The objects rarely have companion planets on nearby orbits
3. Nearly one-third of hot Jupiters have orbital paths that are inclined with respect to their star's equator, and several planets in the population rotate in the opposite direction to the star

In [ ]:
#There is an excess of objects that have orbital periods shorter than about 10 days and masses similar to that of Jupiter
df_hot_jup = df.query('0.36 < pl_bmassj <= 11.8 and pl_orbper < 7')
df_hot_jup.head()

In [ ]:
from matplotlib import pylab as pl
%pylab inline
matplotlib.rcParams.update({'font.size': 16})

#plot all
pl.plot(df['pl_orbsmax'],df['pl_bmassj'],'ro', alpha=0.1)
#plot Hot Jupiter
pl.plot(df_hot_jup['pl_orbsmax'],df_hot_jup['pl_bmassj'],'bo')
pl.xlabel('Semi-major axis [AU]')
pl.ylabel('Planet Mass [$M_{Jup}$]')
pl.xlim([5e-3, 1e4])
#pl.title('Hot Jupiters')
#scale in log to avoid over-crowding
pl.xscale('log')
pl.yscale('log')
#indicate E and J
pl.plot(a_E/a_E,M_E/M_J,'ko', markersize=8, label='Earth')
pl.text(a_E/a_E+0.5,M_E/M_J,'Earth')
pl.plot(a_J/a_E,M_J/M_J,'ko', markersize=8, label='Jupiter')
pl.text(a_J/a_E+2,M_J/M_J,'Jupiter')
#pl.plot(a_N/a_E,M_N/M_J,'k^', markersize=15, label='Neptune')
label1=str('confirmed \nexoplanets ({})'.format(len(df)))
label2=str('Hot Jupiter ({})'.format(len(df_hot_jup)))
leg = pl.legend([label1,label2], fancybox=False, loc=4, numpoints = 1, fontsize=12)
leg.get_frame().set_alpha(0.5)
pl.show()

# Giant Planet near the Snow Line

In [ ]:
#There is an excess of objects that have orbital periods shorter than about 10 days and masses similar to that of Jupiter
df_SL = df.query('0.36 < pl_bmassj <= 11.8 and 300 < pl_orbper < 1460') #4 years
df_SL.head()

# mini-Neptunes and Super-Earth

In [ ]:
M_E = 5.972e24
M_J = 1.898e27
M_mNep1 = 3*M_E/M_J
M_mNep2 = 10*M_E/M_J

In [ ]:
M_mNep1, M_mNep2

In [ ]:
df_mini_Nep = df.query('0094 < pl_bmassj <= 0315 and 300 < pl_orbper < 1460') #4 years
#df_super_Earth = 

In [ ]:
from matplotlib import pylab as pl
%pylab inline
matplotlib.rcParams.update({'font.size': 16})

#plot all
pl.plot(df['pl_orbsmax'],df['pl_bmassj'],'go', alpha=0.1)
#plot Hot Jupiter
pl.plot(df_hot_jup['pl_orbsmax'],df_hot_jup['pl_bmassj'],'ro')
pl.plot(df_SL['pl_orbsmax'],df_SL['pl_bmassj'],'go')
pl.xlabel('Semi-major axis [AU]')
pl.ylabel('Planet Mass [$M_{Jup}$]')
pl.xlim([5e-3, 1e4])
#pl.title('Hot Jupiters')
#scale in log to avoid over-crowding
pl.xscale('log')
pl.yscale('log')
#vetical lines
#pl.axvline(300^(2/3), c='k',ls='--')
#pl.axvline(1460^(2/3), c='k',ls='--')
#indicate E and J
pl.plot(a_E/a_E,M_E/M_J,'ko', markersize=8, label='Earth')
pl.text(a_E/a_E+0.5,M_E/M_J,'Earth')
pl.plot(a_J/a_E,M_J/M_J,'ko', markersize=8, label='Jupiter')
pl.text(a_J/a_E+2,M_J/M_J,'Jupiter')
#pl.plot(a_N/a_E,M_N/M_J,'k^', markersize=15, label='Neptune')
label1=str('confirmed \nexoplanets ({})'.format(len(df)))
label2=str('Hot Jupiter ({})'.format(len(df_hot_jup)))
label3=str('GP near Snow Line ({})'.format(len(df_SL)))
leg = pl.legend([label1,label2,label3], fancybox=False, loc=4, numpoints = 1, fontsize=12)
leg.get_frame().set_alpha(0.5)
pl.show()

## Histograms

Bayesian Blocks is a dynamic histogramming method which optimizes one of several possible fitness functions to determine an optimal binning for data, where the bins are not necessarily uniform width. The astroML implementation is based on [1]. For more discussion of this technique, see the blog post at [2].

See [astroml page](http://www.astroml.org/user_guide/density_estimation.html) for density estimation using histogram recipes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astroML.plotting import hist
#x = np.random.normal(size=1000)
x = df['pl_orbsmax'].dropna().values

fig = plt.figure(figsize=(10, 4))
fig.subplots_adjust(left=0.1, right=0.95, bottom=0.15)

for bins, title, subplot in zip(['knuth', 'blocks'],
                                ["Knuth's rule", 'Bayesian blocks'],
                                [121, 122]):
    ax = fig.add_subplot(subplot)

    # plot a standard histogram in the background, with alpha transparency
    hist(np.log(x), bins=100, histtype='stepfilled',
         alpha=0.2, normed=True, label='standard histogram')

    # plot an adaptive-width histogram on top
    hist(np.log(x), bins=bins, ax=ax, color='black',
         histtype='step', normed=True, label=title)

    ax.legend(prop=dict(size=12))
    ax.set_xlabel('$\log$ Semi-major axis (AU)')
    ax.set_ylabel('Count')

plt.show()

A comparison of different density estimation methods for two simulated one-dimensional data sets (same as in figure 6.5). Density estimators are Bayesian blocks (Section 5.7.2), KDE (Section 6.1.1), and a Gaussian mixture model. In the latter, the optimal number of Gaussian components is chosen using the BIC (eq. 5.35). In the top panel, GMM solution has three components but one of the components has a very large width and effectively acts as a nearly flat background.

See http://www.astroml.org/book_figures/chapter6/fig_GMM_density_estimation.html

# Super-Earths and mini-Neptunes

In [ ]:
pl.plot(df_transit['pl_radj'],df_transit['pl_orbper'],'bo', label='transit ({})'.format(np.count_nonzero(df_transit['pl_radj']>0)))
#pl.plot(df_RV['pl_radj'],df_RV['pl_orbper'],'ro', label='RV ({})'.format(np.count_nonzero(df_RV['pl_radj']>0)))
#pl.plot(df_TTV['pl_radj'],df_TTV['pl_orbper'],'go', label='TTV ({})'.format(np.count_nonzero(df_TTV['pl_radj']>0)))
#pl.plot(df_DI['pl_radj'],df_DI['pl_orbper'],'mo', label='direct imaging ({})'.format(np.count_nonzero(df_DI['pl_radj']>0)))
pl.xlabel('Planet Radius [AU]')
pl.ylabel('Orbital Period [days]')
pl.title('Transiting Super-Earths')
#pl.xlim([5e-3, 1e5])
pl.xscale('log')
pl.yscale('log')
pl.plot(R_E/R_J,P_E,'k^', markersize=12, label='Earth')
pl.plot(R_N/R_J,P_N,'k*', markersize=15, label='Neptune')
pl.legend(loc=2, numpoints = 1)

## Photospheric temperature - Discovery Year

In [ ]:
import matplotlib.pyplot as pl
%matplotlib inline
pl.scatter(df['pl_disc'], df['st_teff'])
pl.ylabel('Photospheric temperature')
pl.xlabel('Year of discovery')
#change to logarithmic scale
#pl.xscale('log')
pl.yscale('log')
#pl.legend(loc='best', numpoints = 1, fontsize=12, handlelength=0.5, fancybox=True, framealpha=0.5)
pl.show()

## Planet - (Stellar) metallicity correlation

In [ ]:
import matplotlib.pyplot as pl
%matplotlib inline
pl.hist(df['st_metfe'], bins=100)
pl.ylabel('Frequency')
pl.xlabel('Metallicity [Fe/H]')
#pl.legend(loc='best', numpoints = 1, fontsize=12, handlelength=0.5, fancybox=True, framealpha=0.5)
pl.show()

# Scatter matrix

In [ ]:
from pandas.tools.plotting import scatter_matrix

scatter_matrix(df[['pl_radj', 'pl_bmassj', 'pl_orbsmax']], alpha=0.2, figsize=(6, 6), diagonal='kde');

Another powerful python library is called [seaborn](http://seaborn.pydata.org/). It offers more advanced plotting capabilities so be sure to learn how to use them. The following is an example.

## Pairplots
This is a powerful way to survey relationship among variables. A particular scattered pairplot implies no correlation while a distribution with a definite slope implies otherwise. Be sure to try varying the keyword parameters.

In [ ]:
import seaborn as sb

variables=df[['pl_radj', 'pl_bmassj', 'pl_dens', 'pl_orbsmax',"st_mass","st_teff","st_rad"]].dropna()
sb.pairplot(variables, diag_kind="kde", plot_kws=dict(s=50, edgecolor="b", linewidth=1), diag_kws=dict(shade=True));
#, hue="pl_discmethod", markers="+"

# What stellar properties are good predictor of hosting planets?

In [ ]:
labels = df.[['[pl_]','']]
features = df[['st_','st_']],

#transform the strings "male" and "female" into binary variables
pd.get_dummies(features).head()

In [ ]:
#1. scale stellar params
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
#scaler.fit(df[['pl_bmassj','pl_radj']].dropna())

In [ ]:
plt.scatter(, c=y, linewidths=0, s=30)
plt.xlabel("feature 1")
plt.ylabel("feature 2");

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)